In [1]:
from collections import defaultdict
import random

import mip
import json
import networkx as nx

# Multicommodity flow

instance: Digraphs `(G, H)`, `u: E(G) -> R_+`, `b: E(H) -> R_+`

task: find `flow: E(H) -> E(G) -> R_+` such that `flow (s, t)` is a s-t-flow of value `b(s, t)` and such that `sum_f flow f e <= u e`

In [2]:
def multicommodity_flow(g, h, u, b):
    
    model = mip.Model()
    
    flow = {(f, e): model.add_var(f"flow-{f}-{e}", lb=0, obj=1) for e in set(g.edges()) for f in h.edges()}

    for e in g.edges():
        model.add_constr(mip.quicksum(flow[f, e] for f in h.edges()) <= u[e])

    for f in h.edges():
        (s, t) = f
        
        for v in g.nodes():
            model.add_constr(
                  mip.quicksum(flow[f, e] for e in set(g.out_edges(v)))
                - mip.quicksum(flow[f, e] for e in set(g.in_edges(v)))
                == (1 if v == s else 0) * b[f] + (-1 if v == t else 0) * b[f])
    
    model.optimize()
    
    
    
    return model.status, {k: v.x for k, v in flow.items() if v.x is not None and v.x > 0}


In [3]:
while True:
    g = nx.scale_free_graph(100)
    h = nx.empty_graph(100, create_using=nx.DiGraph)

    u = defaultdict(int)
    for e in g.edges():
        u[e] += 1
    u = dict(u)

    b = dict()
    for _ in range(2):
        while True:
            x = random.randrange(100)
            y = random.randrange(100)

            if x == y :
                continue
            if (x, y) in b:
                continue

            break

        b[x, y] = 2
        h.add_edge(x, y)

    res = multicommodity_flow(g, h, u, b)
    
    if len(res) == 0:
        continue
        
    break

res


InterfacingError: Gurobi environment could not be loaded, check your license.

# factorio routing problem as an multicommodity flow problem


.


In [4]:
# the problem is to generate the graph G. Nodes will be generated as strings with multiple layers:

from itertools import product
from collections import defaultdict

g = nx.DiGraph()

n = 10

drawing = {}

for x, y in product(range(n), repeat=2):
    g.add_node(f"{x}-{y}")
    drawing[f"{x}-{y}"] = (x, y)

for x, y in product(range(n), repeat=2):
    
    if x == 5 and 3 <= y <= 7:
        continue

    if x != n-1:
        g.add_edge(f"{x}-{y}", f"{x+1}-{y}")
        g.add_edge(f"{x+1}-{y}", f"{x}-{y}")
    if y != n-1:
        g.add_edge(f"{x}-{y}", f"{x}-{y+1}")
        g.add_edge(f"{x}-{y+1}", f"{x}-{y}")

b = {
    ("0-1", "9-9"): 2,
    ("0-5", "9-6"): 2,
    ("5-5", "5-5"): 10
}

h = nx.DiGraph()

for u, v in b.keys():
    h.add_edge(u, v)

u = defaultdict(lambda: 1)

(status, res) = multicommodity_flow(g, h, u, b)
print(status)

taken = set(map(lambda x: x[1], res.keys()))

colors_iter = iter(["red", "blue", "green"])
colors = defaultdict(lambda: next(colors_iter))
edge_colors_d = defaultdict(lambda: "black", {v: colors[k] for k, v in res})

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
nx.draw(g, drawing, edge_color=[edge_colors_d[e] for e in g.edges()], node_color="black", width=3, connectionstyle="arc3,rad=0.1")

InterfacingError: Gurobi environment could not be loaded, check your license.

In [5]:
def multicommodity_flow_incompat(g, u, b, i=[]):
    
    model = mip.Model()
    model.emphasis = 1
    model.max_mip_gap = 0.1
    
    flow = {(f, e): model.add_var(f"flow-{f}-{e}", lb=0, obj=1, var_type=mip.BINARY) for e in set(g.edges()) for f in b.keys()}

    for e in g.edges():
        model.add_constr(mip.quicksum(flow[f, e] for f in b.keys()) <= u[e])
    
    for s in i:
        model.add_constr(mip.quicksum(flow[f, e] for f in b.keys() for e in s) <= 1)

    for f in b.keys():
        (s, t) = f
        
        for v in g.nodes():
            model.add_constr(
                  mip.quicksum(flow[f, e] for e in set(g.out_edges(v)))
                - mip.quicksum(flow[f, e] for e in set(g.in_edges(v)))
                == (1 if v == s else 0) * b[f] + (-1 if v == t else 0) * b[f])
    
    model.optimize()
    
    
    
    return model.status, {k: v.x for k, v in flow.items() if v.x is not None and v.x > 0}

In [7]:
from itertools import product
from matplotlib import pyplot as plt
import factorio

n = 10

g = nx.DiGraph()

shift = {"n": (0, .4), "w": (-.4, 0), "e": (.4, 0), "s": (0, -.4)}

drawing = {}
for x, y, d in product(range(n), range(n), "nwes"):
    g.add_node(f"{x}-{y}-{d}")
    
    dx, dy = shift[d]

    drawing[f"{x}-{y}-{d}"] = (x+dx, y+dy)

incompatibilities = []
for x, y in product(range(n), repeat=2):
    if y != n-1:
        g.add_edge(f"{x}-{y}-n", f"{x}-{y+1}-s")
        g.add_edge(f"{x}-{y+1}-s", f"{x}-{y}-n")
    
    if x != n-1:
        g.add_edge(f"{x}-{y}-e", f"{x+1}-{y}-w")
        g.add_edge(f"{x+1}-{y}-w", f"{x}-{y}-e")
    
    s = []
    for d1, d2 in product("nwes", repeat=2):
        g.add_edge(f"{x}-{y}-{d1}", f"{x}-{y}-{d2}")
        s.append((f"{x}-{y}-{d1}", f"{x}-{y}-{d2}"))

    incompatibilities.append(set(s))

b = {("0-0-s", "9-9-n"): 1}
u = defaultdict(lambda: 1)

(status, res) = multicommodity_flow_incompat(g, u, b, incompatibilities)

print(status)

taken = set(map(lambda x: x[1], res.keys()))

colors_iter = iter(["red", "blue", "green"])
colors = defaultdict(lambda: next(colors_iter))
edge_colors_d = defaultdict(lambda: "black", {v: colors[k] for k, v in res})

# plt.figure(figsize=(10, 10))
# nx.draw(g, drawing, edge_color=[edge_colors_d[e] for e in g.edges()], node_color="black", width=3, connectionstyle="arc3,rad=0.3", node_size=20)



directions = {"n": 4, "s": 0, "e": 2, "w": 6}

entities = []
for i, (u, v) in enumerate(taken, start=1):
    if u.split("-")[:2] != v.split("-")[:2]:
        continue
    
    x, y = map(int, v.split("-")[:2])
    d = v.split("-")[2]
    
    entities.append({"entity_number": i, "name": "fast-transport-belt", "position": {"x": x+.5, "y": y+.5}, "direction": directions[d]})

factorio.generate({
  "blueprint": {
    "icons": [
      {
        "signal": {
          "type": "item",
          "name": "transport-belt"
        },
        "index": 1
      }
    ],
    "entities": entities,
    "item": "blueprint",
    "version": 281479273427424
  }
})


OptimizationStatus.OPTIMAL


'0eJyllt1qhDAQhV9lyXV3MT8a7auUUnQ3LQGNkmRLRXz3jiJauq7E8S6ZCfPlyJmJHSnKu2qsNp68njqir7VxsHrriNNfJi/HqG8bBQuivarIy4mYvBr33ubGNbX150KVnvSQ0uamfiBH+3fYKeO112qqOO7aD3OvCmUhxP6U+sydP/+rB+mmdlCgNuM1hrrJJYZwCyt+iQfgTVt1nY4ICDxiOAITzRj2iGGrGHFITRyqJj6ESULVJAgM3/po62pSBCbessC6mgyBkTMmDVVDMVZLZ04WKodiTJBtcZ7owbhAbpntCUceatEomIPxAd9vNxYd0kND9TCMHrZ/tHGMnmXoiOBJjXkRKEIPpk8XX8tgPZj+Efv9xjH9I7fnzvB8j+89lFz+ESD4raybDqVUyIxJLpgUTPT9L9qGkck='

In [8]:
from itertools import product
from matplotlib import pyplot as plt
import factorio
import numpy as np

n = 30

g = nx.DiGraph()

shift = {"n": (0, .4), "w": (-.4, 0), "e": (.4, 0), "s": (0, -.4)}
opposite = {"n": "s", "w": "e", "e": "w", "s": "n"}

drawing = {}
for x, y, d in product(range(n), range(n), "nwes"):
    g.add_node(f"{x}-{y}-{d}")
    
    dx, dy = shift[d]

    drawing[f"{x}-{y}-{d}"] = (x+dx, y+dy)


for x, y in product(range(n), repeat=2):
    if y < n-1:
        g.add_edge(f"{x}-{y}-n", f"{x}-{y+1}-s")
        g.add_edge(f"{x}-{y+1}-s", f"{x}-{y}-n")
    
    if x != n-1:
        g.add_edge(f"{x}-{y}-e", f"{x+1}-{y}-w")
        g.add_edge(f"{x+1}-{y}-w", f"{x}-{y}-e")

incompatibilities = defaultdict(list)
for x, y in product(range(n), repeat=2):
    s = []
    for d1, d2 in product("nwes", repeat=2):
        g.add_edge(f"{x}-{y}-{d1}", f"{x}-{y}-{d2}")
        incompatibilities[x, y].append((f"{x}-{y}-{d1}", f"{x}-{y}-{d2}"))
    
    if y < n-2:
        g.add_edge(f"{x}-{y}-s", f"{x}-{y+2}-n")
        g.add_edge(f"{x}-{y+2}-n", f"{x}-{y}-s")

        incompatibilities[x, y].append((f"{x}-{y}-s", f"{x}-{y+2}-n"))
        incompatibilities[x, y+2].append((f"{x}-{y}-s", f"{x}-{y+2}-n"))
        incompatibilities[x, y].append((f"{x}-{y+2}-n", f"{x}-{y}-s"))
        incompatibilities[x, y+2].append((f"{x}-{y+2}-n", f"{x}-{y}-s"))
    

    if x < n-2:
        g.add_edge(f"{x}-{y}-w", f"{x+2}-{y}-e")
        g.add_edge(f"{x+2}-{y}-e", f"{x}-{y}-w")

        incompatibilities[x+2, y].append((f"{x}-{y}-w", f"{x+2}-{y}-e"))
        incompatibilities[x, y].append((f"{x}-{y}-w", f"{x+2}-{y}-e"))
        incompatibilities[x, y].append((f"{x+2}-{y}-e", f"{x}-{y}-w"))
        incompatibilities[x+2, y].append((f"{x+2}-{y}-e", f"{x}-{y}-w"))


b = {
    ("0-0-s", "9--e"): 1,
    ("0-1-w", "9-9-e"): 1,
    ("0-2-w", "9-8-e"): 1,
    ("0-3-w", "9-7-e"): 1,
    ("0-4-w", "9-6-e"): 1,
    ("0-5-w", "9-5-e"): 1,
}
u = defaultdict(lambda: 1)

(status, res) = multicommodity_flow_incompat(g, u, b, incompatibilities.values())

print(status)

taken = set(map(lambda x: x[1], res.keys()))

colors_iter = iter(["red", "blue", "green", "yellow", "cyan", "brown", "darkorchid"])
colors = defaultdict(lambda: next(colors_iter))
edge_colors_d = defaultdict(lambda: (0, 0, 0, 0.1), {v: colors[k] for k, v in res})





directions = {"n": 4, "s": 0, "e": 2, "w": 6}

entities = []
for i, (u, v) in enumerate(taken, start=1):
    if u.split("-")[:2] != v.split("-")[:2]:
        continue
    
    x, y = map(int, v.split("-")[:2])
    d = v.split("-")[2]
    
    entities.append({"entity_number": i, "name": "fast-transport-belt", "position": {"x": x+.5, "y": y+.5}, "direction": directions[d]})

for i, (u, v) in enumerate(taken, start=i+1):
    if tuple(np.abs(np.array(list(map(int, u.split("-")[:2]))) - np.array(list(map(int, v.split("-")[:2]))))) not in [(0, 2), (2, 0)]:
        continue
    
    x1, y1 = map(int, u.split("-")[:2])
    x2, y2 = map(int, v.split("-")[:2])
    d1 = u.split("-")[2]
    d2 = v.split("-")[2]
    
    entities.append({"entity_number": i, "name": "fast-underground-belt", "position": {"x": x1+.5, "y": y1+.5}, "direction": directions[d2], "type": "input"})
    entities.append({"entity_number": i, "name": "fast-underground-belt", "position": {"x": x2+.5, "y": y2+.5}, "direction": directions[d2], "type": "output"})




print(factorio.generate({
  "blueprint": {
    "icons": [
      {
        "signal": {
          "type": "item",
          "name": "transport-belt"
        },
        "index": 1
      }
    ],
    "entities": entities,
    "item": "blueprint",
    "version": 281479273427424
  }
}))





OptimizationStatus.INFEASIBLE
0eJxFjUEKwzAMBL8SdG4OcQ1u+5WQg9OKInBkY6slwfjvVXzpbWcWrSqs4YMpEws8hgr0jFw0zRUKvdmHbuVIqAFIcIPLAOy3zpI9lxSzjCsGgaYV8Qt37aa2KCELCWFfPLkP6OH/qcov5kKR1ZvbZN3duKs1zhrb2g8aijGz


In [9]:
from itertools import product
from matplotlib import pyplot as plt
import factorio
import numpy as np

n = 30

g = nx.DiGraph()

shift = {"n": (0, .4), "w": (-.4, 0), "e": (.4, 0), "s": (0, -.4)}
opposite = {"n": "s", "w": "e", "e": "w", "s": "n"}

drawing = {}
for x, y, d in product(range(n), range(n), "nwes"):
    g.add_node(f"{x}-{y}-{d}")
    
    dx, dy = shift[d]

    drawing[f"{x}-{y}-{d}"] = (x+dx, y+dy)

# tile linking
for x, y in product(range(n), repeat=2):
    if y < n-1:
        g.add_edge(f"{x}-{y}-n", f"{x}-{y+1}-s")
        g.add_edge(f"{x}-{y+1}-s", f"{x}-{y}-n")
    
    if x != n-1:
        g.add_edge(f"{x}-{y}-e", f"{x+1}-{y}-w")
        g.add_edge(f"{x+1}-{y}-w", f"{x}-{y}-e")

        
# belt placements
incompatibilities = defaultdict(list)
for x, y in product(range(n), repeat=2):
    s = []
    
    # normal belts
    for d1, d2 in product("nwes", repeat=2):
        g.add_edge(f"{x}-{y}-{d1}", f"{x}-{y}-{d2}")
        incompatibilities[x, y].append((f"{x}-{y}-{d1}", f"{x}-{y}-{d2}"))
    
    # underground belts
    
    for k in range(2, 5):
        if y < n-k:
            g.add_edge(f"{x}-{y}-s", f"{x}-{y+k}-n")
            g.add_edge(f"{x}-{y+k}-n", f"{x}-{y}-s")

            incompatibilities[x, y].append((f"{x}-{y}-s", f"{x}-{y+k}-n"))
            incompatibilities[x, y+k].append((f"{x}-{y}-s", f"{x}-{y+k}-n"))
            incompatibilities[x, y].append((f"{x}-{y+k}-n", f"{x}-{y}-s"))
            incompatibilities[x, y+k].append((f"{x}-{y+k}-n", f"{x}-{y}-s"))
            
            for j in range(0, k+1):
                incompatibilities[x, y+j, "v"].append((f"{x}-{y}-s", f"{x}-{y+k}-n"))
                incompatibilities[x, y+j, "v"].append((f"{x}-{y+k}-n", f"{x}-{y}-s"))

        if x < n-k:
            g.add_edge(f"{x}-{y}-w", f"{x+k}-{y}-e")
            g.add_edge(f"{x+k}-{y}-e", f"{x}-{y}-w")

            incompatibilities[x+k, y].append((f"{x}-{y}-w", f"{x+k}-{y}-e"))
            incompatibilities[x, y].append((f"{x}-{y}-w", f"{x+k}-{y}-e"))
            incompatibilities[x, y].append((f"{x+k}-{y}-e", f"{x}-{y}-w"))
            incompatibilities[x+k, y].append((f"{x+k}-{y}-e", f"{x}-{y}-w"))
            
            for j in range(0, k+1):
                incompatibilities[x+j, y, "h"].append((f"{x}-{y}-w", f"{x+k}-{y}-e"))
                incompatibilities[x+j, y, "h"].append((f"{x+k}-{y}-e", f"{x}-{y}-w"))


b = {
    ("0-0-s", "29-29-e"): 1,
    ("0-29-w", "29-0-e"): 1,
    ("0-1-w", "29-1-e"): 1,
    ("0-2-w", "29-2-e"): 1,
    ("0-3-w", "29-3-e"): 1,
    ("0-4-w", "29-4-e"): 1,
    ("0-5-w", "29-5-e"): 1,
}
u = defaultdict(lambda: 1)

(status, res) = multicommodity_flow_incompat(g, u, b, incompatibilities.values())

print(status)

taken = set(map(lambda x: x[1], res.keys()))

colors_iter = iter(["red", "blue", "green", "yellow", "cyan", "brown", "darkorchid"])
colors = defaultdict(lambda: next(colors_iter))
edge_colors_d = defaultdict(lambda: (0, 0, 0, 0.1), {v: colors[k] for k, v in res})





directions = {"n": 4, "s": 0, "e": 2, "w": 6}

entities = []
for i, (u, v) in enumerate(taken, start=1):
    if u.split("-")[:2] != v.split("-")[:2]:
        continue
    
    x, y = map(int, v.split("-")[:2])
    d = v.split("-")[2]
    
    entities.append({"entity_number": i, "name": "fast-transport-belt", "position": {"x": x+.5, "y": y+.5}, "direction": directions[d]})

for i, (u, v) in enumerate(taken, start=i+1):
    if tuple(np.abs(np.array(list(map(int, u.split("-")[:2]))) - np.array(list(map(int, v.split("-")[:2]))))) not in [(0, k) for k in range(2, 5)] + [(k, 0) for k in range(2, 5)]:
        continue
    
    x1, y1 = map(int, u.split("-")[:2])
    x2, y2 = map(int, v.split("-")[:2])
    d1 = u.split("-")[2]
    d2 = v.split("-")[2]
    
    entities.append({"entity_number": i, "name": "fast-underground-belt", "position": {"x": x1+.5, "y": y1+.5}, "direction": directions[d2], "type": "input"})
    entities.append({"entity_number": i, "name": "fast-underground-belt", "position": {"x": x2+.5, "y": y2+.5}, "direction": directions[d2], "type": "output"})




print(factorio.generate({
  "blueprint": {
    "icons": [
      {
        "signal": {
          "type": "item",
          "name": "transport-belt"
        },
        "index": 1
      }
    ],
    "entities": entities,
    "item": "blueprint",
    "version": 281479273427424
  }
}))


# plt.figure(figsize=(10, 10))
# nx.draw(g, drawing, edge_color=[edge_colors_d[e] for e in g.edges()], node_color="black", width=3, connectionstyle="arc3,rad=0.3", node_size=20)

OptimizationStatus.OPTIMAL
0eJytm91u4jAQhV+lynVbxRMnwfsqq2oFbbaK1AYUwmqrindfg1CJNpY7nDlXJaHk48TO8fyYz2Lzduh2Yz9MxY+7z6J/3g77+OrnZ7HvX4f12/ns9LHr4ouin7r34v6uGNbv5+NpXA/73XacHjbd21Qc41v98NL9je+541M86oapn/rucsXz0cev4fC+6cZ4SmaX+r3eTw//XS++vdvu4wW2w/lrnK5bP9bx9Mfp0+GxPhFf+rF7vvxPGU8kOCsTyK2WIEmCKkSRc1+kcgnySZBvAJC4rKT0vauDjZSQlL53DQIqb+cEZDLMxig17dIk5xajdIhPx/g6buNfzcyrEqh4/PU4DrvD+anjsYOWvT1MGfjiHqvgch1Ob1AOwr0Wnpe+mMYq+lW5GJRjbK9lZ4VLaR3z5JOllI7SvZqeF78w/htH3RmkY2yvZeeFVxDc5ZdxtXSQHkij7q1TvjZoB+FeC89Lb7E7PwtBEtPOa7Xb6QmP92rx2ArjhDPpQTrJ6ipnvfeSCnOV4gl0y8hXoOFQjB6Fc5y+qjG/qbMhulY6CA9aeF46FkvnkyCtcmsOYRKO+WyTT521ykGTL9X0vHYsmJ6vMQabM8MTi7va5TxospT0DYVz0jePBdOzkM6Qs6NwT0nalzWlWw3eEM2i8ECJZj0YUnGkg3CSdNBpOB4P0oXk8TXoNbPsFdeOwoMWnpeOpY8NI6YB2S5f7VUrx2LJ+cL+zeKa1W6nJ7I39dJemzPnxqDdnrZ/kzrmtZtd3pC8ofBASd6WXZxbnzjDAofCOZWqBguqakYsC7IDJZRtsHKBUGoVKJxTq2jsJmsIalD6Nx1nrfjW3ojBtaNwTg+qNSeuFunmxNUkHcwdr1M+tZ1CK90MT0QVpVo6WBycPe2tQbudnlje9OLBbrcwAnkU7imBfAsmj5TsDYVzsrcVZrOUKhXI5hSpVpjTOI5yEE6Sjj1ss9me

## Adding cost to undergounds


In [9]:
def multicommodity_flow_cost(g, u, b, i=[]):
    
    model = mip.Model()
    model.emphasis = 1
    model.max_mip_gap = 0.1
    
    flow = {(f, (u, v)): model.add_var(f"flow-{f}-{(u, v)}", lb=0, obj=d["cost"], var_type=mip.BINARY) for (u, v, d) in (g.edges(data=True)) for f in b.keys()}

    for e in g.edges():
        model.add_constr(mip.quicksum(flow[f, e] for f in b.keys()) <= u[e])
    
    for s in i:
        model.add_constr(mip.quicksum(flow[f, e] for f in b.keys() for e in s) <= 1)

    for f in b.keys():
        (s, t) = f
        
        for v in g.nodes():
            model.add_constr(
                  mip.quicksum(flow[f, e] for e in set(g.out_edges(v)))
                - mip.quicksum(flow[f, e] for e in set(g.in_edges(v)))
                == (1 if v == s else 0) * b[f] + (-1 if v == t else 0) * b[f])
    
    model.optimize(max_seconds=1800)
    
    
    
    return model.status, {k: v.x for k, v in flow.items() if v.x is not None and v.x > 0}

In [10]:
from itertools import product, cycle
from matplotlib import pyplot as plt
import factorio
import numpy as np

n = 10

g = nx.DiGraph()

shift = {"n": (0, .4), "w": (-.4, 0), "e": (.4, 0), "s": (0, -.4)}
opposite = {"n": "s", "w": "e", "e": "w", "s": "n"}

drawing = {}
for x, y, d in product(range(n), range(n), "nwes"):
    g.add_node(f"{x}-{y}-{d}")
    
    dx, dy = shift[d]

    drawing[f"{x}-{y}-{d}"] = (x+dx, y+dy)

# tile linking
for x, y in product(range(n), repeat=2):
    if y < n-1:
        g.add_edge(f"{x}-{y}-n", f"{x}-{y+1}-s", cost=1, type="link")
        g.add_edge(f"{x}-{y+1}-s", f"{x}-{y}-n", cost=1, type="link")
    
    if x != n-1:
        g.add_edge(f"{x}-{y}-e", f"{x+1}-{y}-w", cost=1, type="link")
        g.add_edge(f"{x+1}-{y}-w", f"{x}-{y}-e", cost=1, type="link")

        
# belt placements
incompatibilities = defaultdict(list)
for x, y in product(range(n), repeat=2):
    s = []
    
    # normal belts
    for d1, d2 in product("nwes", repeat=2):
        g.add_edge(f"{x}-{y}-{d1}", f"{x}-{y}-{d2}", cost=1, type="normal")
        incompatibilities[x, y].append((f"{x}-{y}-{d1}", f"{x}-{y}-{d2}"))
    
    # underground belts
    
    for k in range(2, 5):
        if y < n-k:
            g.add_edge(f"{x}-{y}-s", f"{x}-{y+k}-n", cost=10, type="underground")
            g.add_edge(f"{x}-{y+k}-n", f"{x}-{y}-s", cost=10, type="underground")

            incompatibilities[x, y].append((f"{x}-{y}-s", f"{x}-{y+k}-n"))
            incompatibilities[x, y+k].append((f"{x}-{y}-s", f"{x}-{y+k}-n"))
            incompatibilities[x, y].append((f"{x}-{y+k}-n", f"{x}-{y}-s"))
            incompatibilities[x, y+k].append((f"{x}-{y+k}-n", f"{x}-{y}-s"))
            
            for j in range(0, k+1):
                incompatibilities[x, y+j, "v"].append((f"{x}-{y}-s", f"{x}-{y+k}-n"))
                incompatibilities[x, y+j, "v"].append((f"{x}-{y+k}-n", f"{x}-{y}-s"))

        if x < n-k:
            g.add_edge(f"{x}-{y}-w", f"{x+k}-{y}-e", cost=10, type="underground")
            g.add_edge(f"{x+k}-{y}-e", f"{x}-{y}-w", cost=10, type="underground")

            incompatibilities[x+k, y].append((f"{x}-{y}-w", f"{x+k}-{y}-e"))
            incompatibilities[x, y].append((f"{x}-{y}-w", f"{x+k}-{y}-e"))
            incompatibilities[x, y].append((f"{x+k}-{y}-e", f"{x}-{y}-w"))
            incompatibilities[x+k, y].append((f"{x+k}-{y}-e", f"{x}-{y}-w"))
            
            for j in range(0, k+1):
                incompatibilities[x+j, y, "h"].append((f"{x}-{y}-w", f"{x+k}-{y}-e"))
                incompatibilities[x+j, y, "h"].append((f"{x+k}-{y}-e", f"{x}-{y}-w"))


b = {
    ("0-0-w", "9-9-e"): 1,
    ("0-1-w", "9-8-e"): 1,
    ("0-2-w", "9-7-e"): 1,
    ("0-3-w", "9-6-e"): 1,
    ("0-4-w", "9-5-e"): 1,
    ("0-5-w", "9-4-e"): 1,
    ("0-6-w", "9-3-e"): 1,
    # ("0-7-w", "9-2-e"): 1,
    # ("0-8-w", "9-1-e"): 1,
    # ("0-9-w", "9-0-e"): 1,
}
u = defaultdict(lambda: 1)

(status, res) = multicommodity_flow_cost(g, u, b, incompatibilities.values())

print(status)

taken = set(map(lambda x: x[1], res.keys()))

colors_iter = iter(cycle(["red", "blue", "green", "yellow", "cyan", "brown", "darkorchid"]))
colors = defaultdict(lambda: next(colors_iter))
edge_colors_d = defaultdict(lambda: (0, 0, 0, 0.1), {v: colors[k] for k, v in res})





directions = {"n": 4, "s": 0, "e": 2, "w": 6}

entities = []
for i, (u, v) in enumerate(taken, start=1):
    x1, y1 = map(int, u.split("-")[:2])
    x2, y2 = map(int, v.split("-")[:2])
    d1 = u.split("-")[2]
    d2 = v.split("-")[2]
    
    data = g.edges()[u, v]
    
    if "type" not in data:
        continue
    
    if data["type"] == "normal":
        entities.append({"entity_number": i, "name": "transport-belt", "position": {"x": x2+.5, "y": y2+.5}, "direction": directions[d2]})
    
    elif data["type"] == "underground":
        entities.append({"entity_number": i, "name": "underground-belt", "position": {"x": x1+.5, "y": y1+.5}, "direction": directions[d2], "type": "input"})
        entities.append({"entity_number": i, "name": "underground-belt", "position": {"x": x2+.5, "y": y2+.5}, "direction": directions[d2], "type": "output"})
        
    
print(len(entities))

print(factorio.generate({
  "blueprint": {
    "icons": [
      {
        "signal": {
          "type": "item",
          "name": "transport-belt"
        },
        "index": 1
      }
    ],
    "entities": entities,
    "item": "blueprint",
    "version": 281479273427424
  }
}))


OptimizationStatus.OPTIMAL
82
0eJytWttu2zAM/ZXCz22hm3XprwzFkLReYSC1A9sZFgT59ylBEA+1rJFHfYovgU5IHh5SVE7Vdndo9kPbTdXLw6lq3/pujFc/TtXYfnSb3fXpdNw38aJqp+azenyous3n9X4aNt2474fpadvspuocX7Xde/MnvpPn13jXdFM7tc1txevd8Wd3+Nw2w+U7/yz1azNOT1/Wi6/3/RgX6Lvrz7isWz/X8fExXtnn+gL43g7N2+0rIj5YwmgARtxh5BJGJWHMV5hDdMXwMfTxcx1IZYHi/d333f5wdfF3IVsqcn+YVqFrwLfuDiyovrUAjM0xJQ3jARids8YkYaQCcDzfHIkwv+ZHRyLhCXccRfabK2IbOZMlwoM5Pp6ME4r8RsZRAhKImQkOliYQOlChc9qkFvWFaXadqDBEs0uhE1wVVLMX5YBV78hZrxDtN7msX8FB1EUB9iDZ6PkqphcpQcCRfHs00l7NiRfIOEh8Zh4YMg6iyoKPYxC/maxgpXEWfmNqRMoiYsOIQQcqdE6ezKKIk7B1DtsQzS6FTlRcQzUbUReVK0VpVtWI+rusc9M4SNY7vrrUSNZLfs9XL+JD4obIduU0WoLQhgqdo6UtK0ZkUbVlxYhMf4vQf94pair9bdnGl7yFs5hSi5xFRFqC0IYKnaVl2Q6MThdElQ1fxRySZp7f87mykVoqYmkcZIKh+T2SK0tnenyQalYDOFgHIrMCRUvnYuiEZlGbH4/Q0vPd6xG6+FyzkFZ/XzbwouMgMjjTkrzF9mWDKPLgMyA8sHx7AhIfyS8foWwgSR4ZBIQHJmdP+qRECqROKX6ApECYoPgFRIpFpWLu+VLJSlPccuz/qH1OcqUoK50JzqwM5QXSCkt+EkiBZJsEOCMRiwSQBdDBiePrlIROTgK/L5TLoxMS5f03NDjl2AmGkNMNmoarnNkrEq2QUGp+LyUVUnQsf88iocH77LrED